In [ ]:
import pandas as pd
df = pd.read_csv('../data/framingham.csv')
df.head()

df = df.drop(columns=['education'])

df.isnull().sum()

df.shape


# Định nghĩa các cột nhị phân
bin_cols = ["male", "currentSmoker", "prevalentStroke", "prevalentHyp", "diabetes"]
# Điền giá trị thiếu cho các đặc trưng nhị phân bằng giá trị thường gặp nhất (mode)
for col in bin_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)
# Kiểm tra xem có giá trị thiếu nào còn lại không
missing_values = df.isnull().sum()
# Nhập các thư viện cần thiết
# Điền giá trị thiếu cho các đặc trưng số
numeric_cols = ["cigsPerDay", "BPMeds", "totChol", "BMI", "heartRate", "glucose"]
for col in numeric_cols:
    median_val = df[col].median()
    df[col].fillna(median_val)
# Kiểm tra xem có giá trị thiếu nào còn lại không
missing_values = df.isnull().sum()

df['TenYearCHD'].value_counts()

from sklearn.utils import resample
# Tách lớp đa số và lớp thiểu số
df_majority = df[df['TenYearCHD'] == 0]
df_minority = df[df['TenYearCHD'] == 1]
# Tăng cường lớp thiểu số
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # Lấy mẫu với thay thế
                                 n_samples=len(df_majority),    # Để khớp với lớp đa số
                                 random_state=42) # Kết quả có thể tái tạo
# Kết hợp lớp đa số với lớp thiểu số đã tăng cường
df_balanced = pd.concat([df_majority, df_minority_upsampled])

df_balanced['TenYearCHD'].value_counts()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
# Tách các đặc trưng (X) và biến mục tiêu (y)
X = df_balanced.drop(columns=['TenYearCHD'])

y = df_balanced['TenYearCHD']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Tiền xử lí dữ liệu
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Khởi tạo StandardScaler
scaler = StandardScaler()
# Định hình scaler cho dữ liệu huấn luyện và biến đổi cả dữ liệu huấn luyện và dữ liệu kiểm tra
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix


# Định nghĩa danh sách các bộ phân loại
classifiers = [
    RandomForestClassifier(),
    LogisticRegression(),
    SVC(),
    KNeighborsClassifier(),
    GaussianNB(),
]

# Tạo một từ điển để lưu trữ kết quả
results = {}

# Huấn luyện và đánh giá từng bộ phân loại
for clf in classifiers:
    clf_name = clf.__class__.__name__
    clf.fit(X_train_scaled, y_train)   # Train_scaled nếu dùng chưa giảm chiều
    y_pred = clf.predict(X_test_scaled)

    # Tính toán độ chính xác
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{clf_name} Accuracy: {accuracy}")

    # Báo cáo phân loại
    print(f"Classification Report for {clf_name}:")
    print(classification_report(y_test, y_pred))

    # Ma trận nhầm lẫn
    print(f"Confusion Matrix for {clf_name}:")
    print(confusion_matrix(y_test, y_pred))
    print("="*50)
    
    
# Định nghĩa một danh sách các bộ phân loại
classifiers = [
    RandomForestClassifier(),
    LogisticRegression(),
    SVC(),
    KNeighborsClassifier(),
    GaussianNB(),
]

# Tạo một DataFrame để lưu trữ kết quả
results = []

# Huấn luyện và đánh giá từng bộ phân loại
for clf in classifiers:
    clf_name = clf.__class__.__name__  # Lấy tên lớp của bộ phân loại
    clf.fit(X_train_scaled, y_train)  # Huấn luyện bộ phân loại
    y_pred = clf.predict(X_test_scaled)  # Dự đoán trên tập kiểm tra

    # Tính toán các chỉ số đánh giá
    accuracy = accuracy_score(y_test, y_pred)  # Tính độ chính xác
    report = classification_report(y_test, y_pred, output_dict=True)  # Tạo báo cáo phân loại
    f1_score = report['weighted avg']['f1-score']  # Tính điểm F1
    precision = report['weighted avg']['precision']  # Tính độ chính xác
    recall = report['weighted avg']['recall']  # Tính độ nhạy

    # Thêm kết quả vào DataFrame
    results.append({'Model': clf_name, 'Accuracy': accuracy, 'F1-Score': f1_score,
                    'Precision': precision, 'Recall': recall})

# Tạo một DataFrame từ danh sách kết quả
results_df = pd.DataFrame(results)  # Tạo DataFrame từ danh sách kết quả
results_df  # Hiển thị DataFrame


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Khởi tạo RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Huấn luyện RandomForestClassifier
rf_classifier.fit(X_train_scaled, y_train)

# Dự đoán trên tập kiểm tra
y_pred_rf = rf_classifier.predict(X_test_scaled)

# Tính toán độ chính xác
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Độ chính xác của Random Forest Classifier:", accuracy_rf)

# Báo cáo phân loại
print("Báo cáo phân loại cho Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))

# Ma trận nhầm lẫn
print("Ma trận nhầm lẫn cho Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_rf))